<img src="https://radar.community.uaf.edu/wp-content/uploads/sites/667/2021/03/HydroSARbanner.jpg" width="100%" />

<br>
<font size="6"> <b>FIER Daily Flood Forecasting Code</b><img style="padding: 7px" src="https://radar.community.uaf.edu/wp-content/uploads/sites/667/2021/03/UAFLogo_A_647.png" width="170" align="right"/></font>

<br>
<font size="4"> <b> Franz J Meyer, University of Alaska Fairbanks</b> <br>
</font>

This notebooks is developing an algorithm to generate daily flood inundation predictions using time series of Sentinel-1 RTC data and GEOGLoWs river runoff forecasts. 
    
The workflow utilizes information available in the fierpy <a href="https://github.com/SERVIR/fierpy">fierpy</a> GitHub repository.
<hr>

# Load Python Libraries

In [ ]:
from ipyfilechooser import FileChooser
import rioxarray as rxr
import xarray as xr
import fierpy
import glob
import pandas as pd
from pathlib import Path
from ipyfilechooser import FileChooser
import re

In [ ]:
def get_dates(dir_path):
    dates = []
    pths = list(dir_path.glob('*.tif*'))

    for p in pths:
        date_regex = '\d{8}'
        date = re.search(date_regex, str(p))
        if date:
            dates.append(date.group(0))
    return dates

<hr>

# Read sentinel1 time series imagery

In [ ]:
fc = FileChooser('/home/jovyan')
display(fc)

In [ ]:
tiff_dir = Path(fc.selected)
tiffs = list(tiff_dir.glob('*.tif*'))

In [ ]:
times = get_dates(tiff_dir)
times.sort()
times = pd.DatetimeIndex(times)
times.name = "time"

In [ ]:
print(times)

In [ ]:
da = xr.concat([rxr.open_rasterio(f) for f in tiffs], dim=times)

# delete the extra data variable 'band'
da = da.sel(band=1, drop=True)

# rename autogenerated x,y as lon,lat 
da = da.rename({'x': 'lon', 'y': 'lat'})
da

<hr>

# Apply rotated eof process

In [ ]:
reof_ds = fierpy.reof(da, n_modes=4)

<hr>

# Get streamflow data from GeoGLOWS & Select the Days we have Observations

In [ ]:
lat,lon = 11.7122,104.9653
q = fierpy.get_streamflow(lat,lon)
q_sel = fierpy.match_dates(q,da.time)

# apply polynomial to different modes to find best stats

In [ ]:
fit_test = fierpy.find_fits(reof_ds,q_sel,ds)